In [27]:
# 556. В погоне за прибылью https://coderun.yandex.ru/problem/in-pursuit-of-profit

In [28]:
with open('easy_pursuit_of_profit_input.txt') as f:
  k, n, cost, c = map(int, f.readline().split()) # кол-во кварталов (т.е. ПВЗ), кол-во домов, расходы на открытие одного ПВЗ, константа
  data = list(map(lambda x: tuple(map(int, x.replace('\n', '').split())), f.readlines()))

In [29]:
len(data)

17038

In [30]:
data[:10]

[(-10452622, 18779177),
 (-66633062, 80466642),
 (30245517, 97800613),
 (-32888777, -63963290),
 (17630614, 18535706),
 (-52715617, -73701033),
 (-92612139, 88359390),
 (-31019760, -24938408),
 (84780777, 50830753),
 (-32308198, 77777632)]

In [31]:
from sklearn.cluster import KMeans

In [32]:
kmeans = KMeans(n_clusters=k)
kmeans.fit(data)

KMeans(n_clusters=402)

In [33]:
# kmeans.cluster_centers_

In [34]:
import pandas as pd
df = pd.DataFrame(data)
df

,0,1
0,-10452622,18779177
1,-66633062,80466642
2,30245517,97800613
3,-32888777,-63963290
4,17630614,18535706
...,...,...
17033,94060900,-15398571
17034,78295322,760737
17035,-32697415,-15896315
17036,49750691,-74011013


In [35]:
kmeans.labels_

array([222, 344, 384, ..., 348, 396, 172], dtype=int32)

In [36]:
df['cluster'] = kmeans.labels_

In [37]:
df['cluster'].value_counts().sort_index()

,count
cluster,
0,41
1,36
2,54
3,39
4,49
...,...
397,41
398,43
399,44


In [38]:
centroids = pd.DataFrame(kmeans.cluster_centers_)
centroids

,0,1
0,6.278307e+07,-8.323327e+06
1,-4.101889e+07,1.847746e+07
2,-7.775265e+07,-5.728290e+07
3,5.168644e+07,8.441954e+07
4,5.230435e+07,-9.159190e+07
...,...,...
397,-9.025745e+07,-9.282963e+07
398,-2.680449e+07,-4.385358e+07
399,4.403104e+07,-9.472850e+07
400,3.101800e+06,-4.692580e+07


In [39]:
df = pd.merge(left=df, right=centroids, left_on='cluster', right_on=centroids.index)
df

,0_x,1_x,cluster,0_y,1_y
0,-10452622,18779177,222,-1.045262e+07,1.877920e+07
1,-66633062,80466642,344,-6.663307e+07,8.046664e+07
2,30245517,97800613,384,3.024553e+07,9.780058e+07
3,-32888777,-63963290,148,-3.288869e+07,-6.396328e+07
4,17630614,18535706,363,1.763063e+07,1.853573e+07
...,...,...,...,...,...
17033,94060900,-15398571,291,9.406090e+07,-1.539859e+07
17034,78295322,760737,124,7.829523e+07,7.606823e+05
17035,-32697415,-15896315,348,-3.269741e+07,-1.589629e+07
17036,49750691,-74011013,396,4.975071e+07,-7.401103e+07


In [40]:
df.columns = ['x', 'y', 'cluster', 'centroid_x', 'centroid_y']
df.head()

,x,y,cluster,centroid_x,centroid_y
0,-10452622,18779177,222,-1.045262e+07,1.877920e+07
1,-66633062,80466642,344,-6.663307e+07,8.046664e+07
2,30245517,97800613,384,3.024553e+07,9.780058e+07
3,-32888777,-63963290,148,-3.288869e+07,-6.396328e+07
4,17630614,18535706,363,1.763063e+07,1.853573e+07


In [41]:
k_size = df['cluster'].value_counts().sort_index()
k_size

,count
cluster,
0,41
1,36
2,54
3,39
4,49
...,...
397,41
398,43
399,44


In [42]:
df = pd.merge(left=df, right=k_size, on='cluster')
df

,x,y,cluster,centroid_x,centroid_y,count
0,-10452622,18779177,222,-1.045262e+07,1.877920e+07,39
1,-66633062,80466642,344,-6.663307e+07,8.046664e+07,46
2,30245517,97800613,384,3.024553e+07,9.780058e+07,43
3,-32888777,-63963290,148,-3.288869e+07,-6.396328e+07,47
4,17630614,18535706,363,1.763063e+07,1.853573e+07,37
...,...,...,...,...,...,...
17033,94060900,-15398571,291,9.406090e+07,-1.539859e+07,42
17034,78295322,760737,124,7.829523e+07,7.606823e+05,42
17035,-32697415,-15896315,348,-3.269741e+07,-1.589629e+07,44
17036,49750691,-74011013,396,4.975071e+07,-7.401103e+07,30


In [43]:
import numpy as np
df['dist'] = np.sqrt((df['x']-df['centroid_x'])**2 + (df['y']-df['centroid_y'])**2)

In [44]:
df['lvl1'] = cost * (np.power(df['dist'], 1/4)+1)/df['count']

In [45]:
df.head()

,x,y,cluster,centroid_x,centroid_y,count,dist,lvl1
0,-10452622,18779177,222,-1.045262e+07,1.877920e+07,39,22.401292,0.814243
1,-66633062,80466642,344,-6.663307e+07,8.046664e+07,46,12.438146,0.625646
2,30245517,97800613,384,3.024553e+07,9.780058e+07,43,37.297715,0.807273
3,-32888777,-63963290,148,-3.288869e+07,-6.396328e+07,47,92.242447,0.872145
4,17630614,18535706,363,1.763063e+07,1.853573e+07,37,26.888428,0.885716


In [46]:
cluster_sum = df.groupby('cluster')['lvl1'].sum()
cluster_sum

,lvl1
cluster,
0,39.417438
1,31.373658
2,39.141276
3,38.021641
4,41.131657
...,...
397,39.690737
398,36.989332
399,38.859542


In [47]:
with open('556_ans.txt', 'w+') as f:
  c_found = c - np.sum(cluster_sum)
  print(c_found, file=f)
  # print('ans', file=f)
  for i, cl in enumerate(kmeans.cluster_centers_):
    print(*cl, sep=' ', file=f)
  for i, cl in enumerate(kmeans.cluster_centers_):
    print(*df[df['cluster']==i].index, sep=' ', file=f)